# Data Preprocessing

Data preprocessing steps for model training.

## Steps

1. **Download data** 
2. **Convert to Parquet** - `datastats2parquet.py` 
3. **Truncate data** - `truncate.py`  
4. **Split data** - `Splitter.py`

In [ ]:
import sys
from pathlib import Path

sys.path.append('../src')

raw_data_dir = Path('RawData')
# Create data directory if it doesn't exist
raw_data_dir.mkdir(exist_ok=True)

print(f"Raw data directory: {raw_data_dir}")

Raw data directory: RawData
Processed data directory: Data


## Step 1: Download Dataset

In this step, we download the raw dataset.
The following cells will download the required dataset files using wget commands. Make sure you have sufficient disk space available.

In [12]:
%%bash
cd RawData

# Download commands
# wget -nv https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_2015.zip
# wget -nv https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2016.zip
wget -nv https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2016.zip
wget -nv https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2016.zip
wget -nv https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q4_2016.zip
wget -nv https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2017.zip
wget -nv https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2017.zip
wget -nv https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2017.zip
# wget -nv https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q4_2017.zip


2026-02-23 11:37:46 URL:https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2016.zip [292369418/292369418] -> "data_Q2_2016.zip" [1]
2026-02-23 11:38:09 URL:https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2016.zip [322384270/322384270] -> "data_Q3_2016.zip" [1]
2026-02-23 11:38:35 URL:https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q4_2016.zip [337175346/337175346] -> "data_Q4_2016.zip" [1]
2026-02-23 11:38:59 URL:https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2017.zip [339084144/339084144] -> "data_Q1_2017.zip" [1]
2026-02-23 11:39:26 URL:https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2017.zip [387036146/387036146] -> "data_Q2_2017.zip" [1]
2026-02-23 11:39:55 URL:https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2017.zip [426033491/426033491] -> "data_Q3_2017.zip" [1]


## Step 2: Convert Data Statistics to Parquet

The `datastats2parquet.py` script converts raw CSV data files into Parquet format. This step unzips archives and concatenates data. Set `unique_ids` higher than the actual number of drives to ensure all drives are included in the final file.

In [13]:
from preprocessing.datastats2parquet import convert_to_parquet, agg_data


In [14]:
import os
DATA_FOLDER = 'RawData'
EXTRACT_FOLDER = 'tmp'
PARQUET_FOLDER = 'Parquet'
MERGED_FOLDER = 'Merged'
# Step 1: Convert zip archives to parquet format
print("Converting zip archives to parquet format...")
convert_to_parquet(DATA_FOLDER, EXTRACT_FOLDER, PARQUET_FOLDER)

Converting zip archives to parquet format...
Unzipping RawData/data_Q3_2017.zip
Converting to parquet tmp/data_Q3_2017
Removing tmp/data_Q3_2017
Unzipping RawData/data_Q2_2016.zip
Converting to parquet tmp/data_Q2_2016
Removing tmp/data_Q2_2016
Unzipping RawData/data_Q1_2017.zip
Converting to parquet tmp/data_Q1_2017
Removing tmp/data_Q1_2017
Unzipping RawData/data_Q3_2016.zip
Converting to parquet tmp/data_Q3_2016
Removing tmp/data_Q3_2016
Unzipping RawData/data_Q4_2016.zip
Converting to parquet tmp/data_Q4_2016
Removing tmp/data_Q4_2016
Unzipping RawData/data_Q4_2017.zip
Converting to parquet tmp/data_Q4_2017
Removing tmp/data_Q4_2017
Unzipping RawData/data_Q2_2017.zip
Converting to parquet tmp/data_Q2_2017
Removing tmp/data_Q2_2017
Unzipping RawData/data_Q1_2016.zip
Converting to parquet tmp/data_Q1_2016
Removing tmp/data_Q1_2016


In [15]:
# Step 2: Aggregate data from parquet files
print("Aggregating data from parquet files...")
df = agg_data(PARQUET_FOLDER, unique_ids=1_000_000, frequency=1, sample_file='data_Q1_2016.parquet')

Aggregating data from parquet files...


Processing data_Q4_2016.parquet
Processing data_Q1_2016.parquet
Processing data_Q2_2016.parquet
Processing data_Q1_2017.parquet
Processing data_Q2_2017.parquet
Processing data_Q3_2016.parquet
Processing data_Q3_2017.parquet
Processing data_Q4_2017.parquet


In [16]:
# Step 3: Print statistics
print(f"Number of unique serial numbers: {df['serial_number'].unique().shape[0]}")

# Get last records for each serial number
last = df[df['date'] == df.groupby('serial_number')['date'].transform('max')]
print(f"Number of last records: {len(last)}")

# Save merged data to parquet
os.makedirs(MERGED_FOLDER, exist_ok=True)
df.to_parquet(os.path.join(MERGED_FOLDER, 'merged.parquet'))
print("Merged data saved to merged.parquet")

Number of unique serial numbers: 64074
Number of last records: 64074
Merged data saved to merged.parquet


In [17]:
import shutil
shutil.rmtree(PARQUET_FOLDER)

## Step 3: Truncate Data

The `truncate.py` script processes the converted Parquet data to handle truncation. Truncated drives are those that are censored only because the observation period ended (right-censored observations).

In [18]:
from preprocessing.truncate import truncate_observations

truncate_observations(input_file=os.path.join(MERGED_FOLDER, 'merged.parquet'), output_file=os.path.join(MERGED_FOLDER, '2016_2017_trunc.parquet'))

Loading data from Merged/merged.parquet...
Initial failure rate: 0.0001
Found 44204 drives with truncated observations
Removed truncated observations. Remaining drives: 19870
Removed 15 inconsistent drives
Final failure rate: 0.0003
Final dataset size: 8856670 records, 19855 unique drives
Processed data saved to Merged/2016_2017_trunc.parquet


,time,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2016-10-31,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,3000592982016,False,100,0,135,108,127,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2016-10-31,MJ0351YNG9WJSA,Hitachi HDS5C3030ALA630,3000592982016,False,100,0,136,104,126,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2016-10-31,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,3000592982016,False,100,0,136,102,137,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2016-10-31,WD-WMC4N2899475,WDC WD30EFRX,3000592982016,False,200,0,<NA>,<NA>,174,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2016-10-31,MJ0351YNG9Z7LA,Hitachi HDS5C3030ALA630,3000592982016,False,100,0,136,104,123,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8859683,2017-10-15,Z300ZV38,ST4000DM000,4000787030016,False,109,22785032,<NA>,<NA>,91,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8859684,2017-10-15,Z1Z02R5Z,ST4000DX000,4000787030016,False,114,62703537,<NA>,<NA>,91,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8859685,2017-10-15,Z302529F,ST4000DM000,4000787030016,False,118,189776024,<NA>,<NA>,92,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8859686,2017-10-15,Z3025294,ST4000DM000,4000787030016,False,118,193987776,<NA>,<NA>,92,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## Step 4: Split Dataset

The `Splitter.py` script performs the final preprocessing step by applying the complete preprocessing pipeline and splitting the data:

- **Stratified sampling**: Creates balanced train/test splits preserving failure rate distribution (train and test sets do not overlap by drives)
- **Feature engineering**: Applies time transformations, aggregation, and scaling
- **Multiple sample sizes**: Generates datasets with different numbers of observations per drive
- **Quality control**: Removes drives with insufficient data or anomalous patterns

This step produces the final preprocessed datasets ready for survival model training.

In [ ]:
from preprocessing.Splitter import main as split_data

split_data(input_file=os.path.join(MERGED_FOLDER, '2016_2017_trunc.parquet'), data_folder=os.path.join('Data', 'Preprocessed'))

Data loaded
train_n: 20, mean: 20.80127236079617
test_n: 1, mean: 2.0
Total dataset: (8856660, 95), Training set: (330241, 32), Test set: (7938, 32)
test_n: 10, mean: 10.960191484001008
Total dataset: (8856660, 95), Training set: (330241, 32), Test set: (43501, 32)
test_n: 25, mean: 25.738977072310405
Total dataset: (8856660, 95), Training set: (330241, 32), Test set: (102158, 32)
Processing n_samples=20 completed


## Done

Preprocessing completed. Generated files:
- `{sample_size}_train_preprocessed.csv` - training data
- `{sample_size}_{test_size}_test_preprocessed.csv` - test data

Next: use `training_demo.ipynb` for model training.